In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import numpy as np
import string
import json

In [13]:
character_metadata = pd.read_csv('Data/character.metadata.tsv', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'Freebase_movie_ID',
                                                                              'Movie_release_date',
                                                                              'Character_name',
                                                                              'Actor_date_of_birth',
                                                                              'Actor_gender',
                                                                              'Actor_height',
                                                                              'Actor_ethnicity',
                                                                              'Actor_name',
                                                                              'Actor_age_at_movie_release',
                                                                              'Freebase_character/actor_map_ID',
                                                                              'Freebase_character_ID',
                                                                              'Freebase_actor_ID'])

movie_metadata = pd.read_csv('Data/movie.metadata.tsv', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'Freebase_movie_ID',
                                                                              'Movie_name',
                                                                              'Movie_release_date',
                                                                              'Movie_box_office_revenue',
                                                                              'Movie_runtime',
                                                                              'Movie_languages',
                                                                              'Movie_countries',
                                                                              'Movie_genres'])

plot_summaries = pd.read_csv('Data/plot_summaries.txt', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'summary'])

In [16]:
movie_crew = pd.read_csv('Projet/Data/movie_crew.tsv', sep='\t')
name_basics = pd.read_csv('Projet/Data/names_basics.tsv', sep='\t')


In [17]:
movie_crew.head() 

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [18]:
name_basics.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0053137,tt0072308,tt0050419,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0038355,tt0075213,tt0117057"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0072562,tt0078723,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0050986,tt0050976,tt0083922"


In [21]:
movies_directors = movie_crew.merge(name_basics, right_on= ['nconst'], left_on= ['directors'], how='inner')

In [22]:
movies_directors.head()

,tconst,directors,writers,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,tt0000001,nm0005690,\N,nm0005690,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1496763,tt0219560,tt0308254,tt1428455"
1,tt0000005,nm0005690,\N,nm0005690,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1496763,tt0219560,tt0308254,tt1428455"
2,tt0000006,nm0005690,\N,nm0005690,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1496763,tt0219560,tt0308254,tt1428455"
3,tt0000008,nm0005690,\N,nm0005690,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1496763,tt0219560,tt0308254,tt1428455"
4,tt0000036,nm0005690,nm0410331,nm0005690,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1496763,tt0219560,tt0308254,tt1428455"


In [23]:
Unuseful = ['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles', 'writers']

movies_directors.drop(Unuseful, axis=1, inplace = True)

In [26]:
movies_directors.sample(10)

,tconst,directors,nconst,primaryName
1483354,tt20194482,nm0273084,nm0273084,Jorge Fernando
2336868,tt12471224,nm2329298,nm2329298,Hidetaka Miyazaki
4622165,tt4897810,nm7484768,nm7484768,Annemikami Pablo
3978158,tt30025577,nm13091781,nm13091781,Nitin Katkar
4620897,tt6480068,nm7462621,nm7462621,George Nevada
523955,tt0290429,nm0450200,nm0450200,Keshu Ramsay
3881758,tt15535050,nm8509943,nm8509943,Blythe Kala
251556,tt1411323,nm0366443,nm0366443,Michael Hart
2507858,tt11283744,nm10678642,nm10678642,Aadithyan
3147184,tt5390404,nm2088669,nm2088669,Natalie Peri


In [28]:
movies_directors.to_csv("Data/movies_directors.csv")